In [10]:
from WD.datasets import Conditional_Dataset_Zarr
import numpy as np
import torch
import zarr
import torch
from torch.utils.data import Sampler, RandomSampler, BatchSampler, SequentialSampler
import dask

In [19]:
a = zarr.open("/data/compoundx/WeatherDiff/model_input/7E2101_test.zarr")

In [21]:
a.inputs.data.info

Name,/inputs/data
Type,zarr.core.Array
Data type,float32
Shape,"(2917, 1, 32, 64)"
Chunk shape,"(2917, 1, 32, 64)"
Order,C
Read-only,False
Compressor,None
Store type,zarr.storage.DirectoryStore
No. bytes,23896064 (22.8M)
No. bytes stored,23896438 (22.8M)


In [ ]:
def get_conditioning_indices(start_index, conditioning_timesteps, batch_size, max_index):
    max_rel_index = max_index - start_index
    return (np.arange(min(batch_size, max_rel_index))[:,None] + conditioning_timesteps).ravel() + start_index

def get_target_indices(start_index, lead_time, batch_size, max_index):
    max_rel_index = max_index - start_index
    return (np.arange(min(batch_size, max_rel_index))[:,None] + lead_time).ravel() + start_index

In [ ]:
def iterator(zarr_group, start, stop, batch_size, conditioning_timesteps, lead_time):
    # careful when calculating number of samples!
    array_inputs = zarr_group.inputs.data
    array_targets = zarr_group.targets.data
    array_constants = zarr_group.constants.data

    assert stop <= array_inputs.shape[0] - lead_time
    chunk_size = array_inputs.chunks[0]
    effective_batchsize = batch_size - min(conditioning_timesteps) + lead_time
    chunks_per_batch = np.ceil(effective_batchsize / chunk_size)

    starts = np.arange(start, stop, batch_size)
    current_chunk = -1
    
    for j in starts:
        print(j, np.arange(j, min(j+batch_size, stop)))
        if j >= (current_chunk+1) * chunk_size:
            chunks_input = torch.tensor(array_inputs.oindex[np.arange(current_chunk * chunk_size, min((chunks_per_batch + 1 + current_chunk) * chunk_size, stop), dtype=int),:,:,:])
            chunks_targets= torch.tensor(array_targets.oindex[np.arange(current_chunk * chunk_size, min((chunks_per_batch + 1 + current_chunk) * chunk_size, stop), dtype=int),:,:,:])
            current_chunk = j // chunk_size

        input_data = chunks_input[get_conditioning_indices(j, conditioning_timesteps=conditioning_timesteps, batch_size=batch_size, max_index=stop) % chunk_size]
        input_data = input_data.view(-1, len(conditioning_timesteps)*input_data.shape[1], *input_data.shape[2:])
        output_data = chunks_targets[get_target_indices(j, lead_time=lead_time, batch_size=batch_size, max_index=stop) % chunk_size]

        input_data = torch.concatenate((input_data, torch.tensor(array_constants[:])[None,...].repeat(input_data.shape[0], 1, 1, 1)), dim=1)
        yield input_data, output_data


In [ ]:
a = zarr.open("/data/compoundx/WeatherDiff/model_input/9921EA_train.zarr")

In [29]:
for k, j in iterator(a, 0, 100, 8, [0,-1,-2], lead_time=2):
    print(k.shape[0])

NameError: name 'iterator' is not defined

In [1]:
from WD.datasets import Conditional_Dataset_Zarr_Iterable
from torch.utils.data import DataLoader
import torch
import zarr

In [14]:
ds = Conditional_Dataset_Zarr_Iterable(zarr_file_path="/data/compoundx/WeatherDiff/model_input/9921EA_test.zarr", config_file_path="/data/compoundx/WeatherDiff/config_file/9921EA.yml", shuffle_chunks=False, shuffle_in_chunks=True)

In [15]:
dl = DataLoader(ds, batch_size=500)

In [16]:
for input, target in dl:
    print(input.shape[0])

500
500
500
500
500
404


In [5]:
a = zarr.open("/data/compoundx/WeatherDiff/model_input/9921EA_test.zarr")

In [2]:
a = zarr.open("/data/compoundx/WeatherDiff/model_input/ACBB5F_train.zarr")

In [24]:
a.inputs.data.shape[0] / 64

5067.015625